In [1]:
from qat.interop.qiskit import QPUToBackend,AsyncBackendToQPU
from qat.qpus import PyLinalg,get_default_qpu
from qat.plugins import ScipyMinimizePlugin as ScipyMinimizePluginLocal
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal

#%%pycodestyle
from qiskit_nature.drivers import UnitsType
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.algorithms import VQEUCCFactory,GroundStateEigensolver

from qiskit_nature.circuit.library import UCCSD,UCC
from qiskit_nature.circuit.library.initial_states import HartreeFock
from qiskit.algorithms.optimizers import SLSQP,L_BFGS_B,SPSA,COBYLA
from qiskit.utils import QuantumInstance
from qiskit import Aer

import time as time


/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# Set up 3 different backends to test computation time
# backend_remoteQLM = ['remote backend QLM', QPUToBackend(qpu=LinAlg(), optimizer=ScipyMinimizePlugin(method="COBYLA"))]
backend_localQLM = ['local backend QLM', QPUToBackend(qpu=PyLinalg() , optimizer=ScipyMinimizePluginLocal(method="COBYLA"))]
backend_localQiskit = ['local backend qiskit', Aer.get_backend('aer_simulator')]
shots=1024

In [3]:
# Initialize qubit converter
qubit_converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)  # two qubit reduction doesn`t work with Jordan-WignerMapper()
optimizer = COBYLA(maxiter=100)
bond_distance = 0.735
# Molecule
molecule = 'H .0 .0 -{0}; H .0 .0 {0}'

driver = PySCFDriver(atom=molecule.format(bond_distance/2), unit=UnitsType.ANGSTROM, basis='sto3g')
# qmolecule = driver.run()
es_problem = ElectronicStructureProblem(driver,q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True, remove_orbitals=[])])

tl_circuit = TwoLocal(4, ['ry', 'rz'], 'cx',
                      entanglement='full', reps=1)
qmolecule = driver.run()
num_particles = qmolecule.num_alpha,qmolecule.num_beta
hf_init = HartreeFock(qubit_converter = qubit_converter, 
                      num_spin_orbitals = qmolecule.num_molecular_orbitals*2, 
                      num_particles = num_particles )
uc_circuit = UCCSD(qubit_converter = qubit_converter,
                   initial_state=hf_init) #doesnt work with conversion to QLM


ansatz_in_use = uc_circuit

for label,backend in [backend_localQiskit,backend_localQLM]:#, backend_remoteQLM]
    quantum_instance = QuantumInstance(backend=backend, shots=shots,  max_credits=None)
    vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
    calc = GroundStateEigensolver(qubit_converter, vqe_solver)
    start = time.time()
    res = calc.solve(es_problem)
    end = time.time()
    print('Type of backend:',label,'VQE:\t', res.total_energies,',exec_time=', end - start)


# Compute exact solution
from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
exact_calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res_exact = exact_calc.solve(es_problem)
print('Exact:\t', res_exact.total_energies)

Type of backend: local backend qiskit VQE:	 [-1.12099759+0.j] ,exec_time= 2.550531229999251


The measured number of particles 1.486328125 does NOT match the expected number of particles 2!


Type of backend: local backend QLM VQE:	 [0.06614028+0.j] ,exec_time= 458.3527719440008
Exact:	 [-1.13730604+0.j]


In [4]:
# convert only the ansatz and pass the hamiltonian via matrix to Observables)
from qat.interop.qiskit import qiskit_to_qlm
from qat.core import Observable


from qiskit.compiler import transpile

def VQEwrapper_to_stack(groundstate_solver, es_problem, stack):
    #if isinstance(groundstate_solver._solver, MinimumEigensolverFactory):
    # this must be called after transformation.transform
    solver = groundstate_solver._solver.get_solver(es_problem, groundstate_solver._qubit_converter)
    #solver = groundstate_solver.solver
    ansatz_in_use = solver.ansatz
    #backend = solver.quantum_instance.backend
    backend = QPUToBackend(stack.qpu)

    transpiled_circ = transpile(ansatz_in_use.decompose(),
                                basis_gates = backend.configuration().basis_gates,
                                optimization_level = 0)
    qcirc = qiskit_to_qlm(transpiled_circ)    

    # Convert the observable/hamiltonian
    
    fermionic_hamiltonian = es_problem.second_q_ops()[0]
    hamiltonian_spin = groundstate_solver._qubit_converter.convert_match(fermionic_hamiltonian)
    num_qubits = hamiltonian_spin.num_qubits
    # print(num_qubits, hamiltonian_spin)
    job = qcirc.to_job(observable = Observable(num_qubits, matrix=hamiltonian_spin.to_matrix()))  

    # Stack to run locally (seems faster)
    
    start = time.time()
    calculation = stack.submit(job)
    end = time.time()

    #print("Final energy :", calculation.value , ' , exec time =', end - start)      
    # Convert to qiskit results format        
    raw_result = solver.compute_minimum_eigenvalue.__annotations__['return']()   
    raw_result.eigenvalue =  calculation.value
    raw_result.optimal_parameters = solver.ansatz.parameters
    dict_param = {}
    for i,par in enumerate(solver.ansatz.parameters):
        dict_param[par] = calculation.parameter_map[str(par)]
    raw_result.eigenstate = solver._get_eigenstate(dict_param)
    
    result = es_problem.interpret(raw_result)
    return result



#the problem
quantum_instance = QuantumInstance(backend=backend_localQLM[1], shots=shots,  max_credits=None)
vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)

#the stack
stack = ScipyMinimizePluginLocal(method="COBYLA") | get_default_qpu()
start = time.time()
res2 = VQEwrapper_to_stack(groundstate_solver=calc , es_problem=es_problem , stack=stack)
end = time.time()

print('Result',res2.total_energies,' ,exec_time=', end - start)

Result [-1.13730603+0.j]  ,exec_time= 4.4816872920000606


In [6]:
from qiskit.algorithms import VQE
from qiskit.opflow import (
    OperatorBase,
)
import numpy as np
from typing import Optional, List, Callable, Union, Dict, Tuple

class VQE_MyQLM(VQE):
    def get_energy_evaluation(
        self,
        operator: OperatorBase,
        return_expectation: bool = False,
    ) -> Callable[[np.ndarray], Union[float, List[float]]]:
        """Returns a function handle to evaluates the energy at given parameters for the ansatz.

        This is the objective function to be passed to the optimizer that is used for evaluation.

        Args:
            operator: The operator whose energy to evaluate.
            return_expectation: If True, return the ``ExpectationBase`` expectation converter used
                in the construction of the expectation value. Useful e.g. to evaluate other
                operators with the same expectation value converter.


        Returns:
            Energy of the hamiltonian of each parameter, and, optionally, the expectation
            converter.

        Raises:
            RuntimeError: If the circuit is not parameterized (i.e. has 0 free parameters).

        """
        num_parameters = self.ansatz.num_parameters
        if num_parameters == 0:
            raise RuntimeError("The ansatz must be parameterized, but has 0 free parameters.")

        expect_op, expectation = self.construct_expectation(
            self._ansatz_params, operator, return_expectation=True
        )
        def create_job(operator,params):
            ansatz_in_use = self._ansatz.bind_parameters(params)
            
            transpiled_circ = transpile(ansatz_in_use.decompose(),
                                basis_gates = self._quantum_instance.backend.configuration().basis_gates,
                                optimization_level = 0)
            qcirc = qiskit_to_qlm(transpiled_circ)

            job = qcirc.to_job(observable = Observable(operator.num_qubits, matrix=operator.to_matrix()))
            return job

        def energy_evaluation(parameters):
            parameter_sets = np.reshape(parameters, (-1, num_parameters))
            #print(expect_op, expectation)
            # Create dict associating each parameter with the lists of parameterization values for it
            param_bindings = dict(zip(self._ansatz_params, parameter_sets.transpose().tolist()))

            #start_time = time()
            sampled_expect_op = self._circuit_sampler.convert(expect_op, params=param_bindings)
            means = np.real(sampled_expect_op.eval())
            print('their',means,param_bindings, type(expect_op))
            #MODMOD
            # print('evaluation')
            # print(operator)
            # print(sampled_expect_op.num_qubits)
            # print(param_bindings)
            job = create_job(operator,parameters)
            result = self.submit_job(job)
            means = np.real([result.value])
            print('mine',means, parameters)
            #MODMOD

            if self._callback is not None:
                variance = np.real(expectation.compute_variance(self._circuit_sampler.convert(expect_op, params=param_bindings)))
                estimator_error = np.sqrt(variance / self.quantum_instance.run_config.shots)
                for i, param_set in enumerate(parameter_sets):
                    self._eval_count += 1
                    self._callback(self._eval_count, param_set, means[i], estimator_error[i])
            else:
                self._eval_count += len(means)

            #end_time = time()
           

            return means if len(means) > 1 else means[0]

        if return_expectation:
            return energy_evaluation, expectation

        return energy_evaluation


# vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
#the problem
quantum_instance = QuantumInstance(backend=backend_localQiskit[1], shots=shots,  max_credits=None)
vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)

calc.solver._vqe = VQE_MyQLM(ansatz=None,
            quantum_instance=calc.solver._quantum_instance,
            optimizer=calc.solver._optimizer,
            initial_point=calc.solver._initial_point,
            gradient=calc.solver._gradient,
            expectation=calc.solver._expectation,
            include_custom=calc.solver._include_custom)
# calc.solver._vqe.submit_job= 'xxx'

In [7]:
import numpy as np
from qat.plugins import Junction
from qat.qpus import get_default_qpu
from qat.core import Result
from qat.core.plugins import AbstractPlugin





class IterativeExploration(Junction):
    def __init__(self, method, es_problem):
        super(IterativeExploration, self).__init__()
        print(method)
        self.method = method
        self.problem = es_problem

    def run(self, initial_job, meta_data):

        self.method.solver._vqe.submit_job = self.execute

        result = self.method.solve(self.problem)


        print(result.total_energies)
        return Result(value=result.total_energies[0])

# class MyPlugin(AbstractPlugin):
#     def __init__(self, method, es_problem):
#         # super(IterativeExploration, self).__init__()
#         print(method)
#         self.method = method
#         self.problem = es_problem
#     def compile(self, batch, hardware_specs):
#         result = self.method(self.problem)
#         print(result.total_energies)
#         return batch

#     def post_process(self, batch_result):
#         # do something with the execution results
#         for result in batch_result.results:
#             print('Result of size', len(result.raw_data))
#         return batch_result

#     def do_post_processing(self):
#         return False



# Building a simple stack
qpu = get_default_qpu()
stack = IterativeExploration(calc, es_problem) | qpu
# stack = MyPlugin(calc.solve, es_problem) | qpu


# and a simple job
from qat.core import Observable
from qat.lang.AQASM import Program, RY

prog = Program()
qbits = prog.qalloc(1)
prog.apply(RY(prog.new_var(float, r"\beta")), qbits)
job = prog.to_circ().to_job(observable=Observable.sigma_z(0, 1))
#/ and a simple job
start = time.time()
calculation = stack.submit(job)
end = time.time()

print('Result',calculation.value,' ,exec_time=', end - start)

their [-1.84041346] {ParameterVectorElement(t[0]): [0.0], ParameterVectorElement(t[1]): [0.0], ParameterVectorElement(t[2]): [0.0]} <class 'qiskit.opflow.list_ops.summed_op.SummedOp'>
mine [-0.24521829] [0. 0. 0.]
their [-1.29031574] {ParameterVectorElement(t[0]): [1.0], ParameterVectorElement(t[1]): [0.0], ParameterVectorElement(t[2]): [0.0]} <class 'qiskit.opflow.list_ops.summed_op.SummedOp'>
mine [-0.8247304] [1. 0. 0.]
their [-0.55329754] {ParameterVectorElement(t[0]): [1.0], ParameterVectorElement(t[1]): [1.0], ParameterVectorElement(t[2]): [0.0]} <class 'qiskit.opflow.list_ops.summed_op.SummedOp'>
mine [-1.23202272] [1. 1. 0.]
their [-0.90434627] {ParameterVectorElement(t[0]): [1.0], ParameterVectorElement(t[1]): [1.0], ParameterVectorElement(t[2]): [1.0]} <class 'qiskit.opflow.list_ops.summed_op.SummedOp'>
mine [-1.23656469] [1. 1. 1.]
their [-1.47945919] {ParameterVectorElement(t[0]): [1.8181299865515876], ParameterVectorElement(t[1]): [1.5749975733070403], ParameterVectorEleme

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 